In [1]:
import numpy as np
import pandas as pd
import datetime
import copy
import time
import os
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import operator

from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
from datetime import timedelta

tqdm.pandas()

In [2]:
# Edit to point to your MIMIC directory.
dataDirStr = '/Users/gmessier/data/mimic-1.4/'

In [3]:
inputevents_cv_df = pd.read_csv(dataDirStr + "INPUTEVENTS_CV.csv")
inputevents_cv_df.columns = inputevents_cv_df.columns.str.lower()
inputevents_cv_df

/Users/gmessier/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,9,17,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,row_id,subject_id,hadm_id,icustay_id,charttime,itemid,amount,amountuom,rate,rateuom,...,orderid,linkorderid,stopped,newbottle,originalamount,originalamountuom,originalroute,originalrate,originalrateuom,originalsite
0,592,24457,184834.0,205776.0,2193-09-11 09:00:00,30056,100.0,ml,NaN,NaN,...,756654,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,593,24457,184834.0,205776.0,2193-09-11 12:00:00,30056,200.0,ml,NaN,NaN,...,3564075,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,594,24457,184834.0,205776.0,2193-09-11 16:00:00,30056,160.0,ml,NaN,NaN,...,422646,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,595,24457,184834.0,205776.0,2193-09-11 19:00:00,30056,240.0,ml,NaN,NaN,...,5137889,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,596,24457,184834.0,205776.0,2193-09-11 21:00:00,30056,50.0,ml,NaN,NaN,...,8343792,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527930,17422687,27946,169481.0,212466.0,2154-07-16 22:00:00,40864,82.0,ml,NaN,NaN,...,10296124,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17527931,17422688,27946,169481.0,212466.0,2154-07-16 23:00:00,40864,82.0,ml,NaN,NaN,...,10248887,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17527932,17422689,27946,169481.0,212466.0,2154-07-17 00:00:00,40864,NaN,ml,NaN,NaN,...,7418343,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17527933,17422690,30516,132490.0,260091.0,2105-08-25 06:30:00,46046,31.0,ml,NaN,NaN,...,1753414,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


`INPUTEVENTS_CV.parquet` contains input events from the CareVue ICU database. Inputs are any fluids which have been administered to the patient: such as oral or tube feedings or intravenous solutions containing medications. 

In [4]:
print(f"There are {inputevents_cv_df.subject_id.nunique()} patients in the CareVue ICU database with input events")

There are 27124 patients in the CareVue ICU database with input events


`charttime` refers to the time when the measurement was charted, this is recorded by the clinical information system at the bedside. 

`itemid` is the identifier for a single measurement type in the database. Refer to `D_ITEMS` table for exact definitions of each `itemid`.

`orderid` links multiple `itemid`s to the same solution together. For example, when a solution of two different medicines are made and administered at the same time.

In [5]:
c = inputevents_cv_df.itemid.value_counts()[:5]
p = inputevents_cv_df.itemid.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
30013,2557507,14.59
30018,2392372,13.65
30131,924614,5.28
30045,825758,4.71
30025,813242,4.64


`amount` and `amountuom` list the amount of a drug or substance administered to the patients either between the `starttime` and the `endtime`.

`rate` and `rateuom` list the rate at which the `item`: drug or substance was administered to the patient.

`storetime` records the time at which an observation was manually inptut or validated by a member of clinical staff.

`cgid` is the identifier for the caregiver who validated the given measurement (see patients table)

In [6]:
c = inputevents_cv_df.cgid.value_counts().nlargest(5)
p = inputevents_cv_df.cgid.value_counts(normalize=True).mul(100).round(2).nlargest(5)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
21570.0,1180209,6.87
15644.0,126130,0.73
15803.0,105783,0.62
20971.0,100536,0.59
15001.0,96442,0.56


`orderid` links multiple `itemid`s to the same solution together. For example, when a solution of two different medicines are made and administered at the same time.

`linkorderid` links the same order across multiple instantiations: for example, if the rate of delivery for the solution with noradrenaline and normal saline is changed, two new rows which share the same new ORDERID will be generated, but the LINKORDERID will be the same.

`stopped` indicates whether the infusion has been disconnected or continued.

In [7]:
c = inputevents_cv_df.stopped.value_counts()[:5]
p = inputevents_cv_df.stopped.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
D/C'd,241622,50.28
Stopped,204822,42.62
Restart,32439,6.75
NotStopd,1668,0.35


`newbottle` indicates if a new preparation of the solution was hung at the bedside.

In [8]:
c = inputevents_cv_df.newbottle.value_counts(dropna=False)
p = inputevents_cv_df.newbottle.value_counts(normalize=True, dropna=False).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
NaN,17282036,98.6
1.0,245899,1.4


`originalamount`, `originalamountuom`, `originalroute`, `originalrate`, `originalrateuom` , `originalsite` provide information about the solution of medication given by `itemid` when it was first entered into the information system.


In [9]:
c = inputevents_cv_df.originalroute.value_counts(dropna=False)
p = inputevents_cv_df.originalroute.value_counts(normalize=True, dropna=False).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
Intravenous Push,9215677,52.58
IV Drip,4131467,23.57
Intravenous,1252093,7.14
Gastric/Feeding Tube,1066706,6.09
Nasogastric,725631,4.14
Intravenous Infusion,480507,2.74
By Mouth,312249,1.78
Oral,188113,1.07
NaN,106683,0.61
Drip,40709,0.23


In [10]:
c = inputevents_cv_df.originalsite.value_counts(dropna=False)
p = inputevents_cv_df.originalsite.value_counts(normalize=True, dropna=False).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
NaN,17350363,98.99
PO / NG,26917,0.15
Right Int. Jugular,25420,0.15
Right Arm,21529,0.12
UVC,17895,0.10
Left Arm,17321,0.10
Left Subclavian,14674,0.08
Right Subclavian,12182,0.07
Percutaneous Cath,12075,0.07
Left Int. Jugular,5995,0.03
